In [1]:
import sklearn
import pandas as pd
import numpy as np
#from deep_translator import GoogleTranslator
import xgboost as xgb
import matplotlib.pyplot as plt
import re
import gc
import numpy as np
#import text2vec


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#no_included_samples = pd.read_csv('/content/drive/MyDrive/reddit/reddit clean/no_include_sample_comm.csv')

training1 = pd.read_csv('/content/drive/MyDrive/reddit/balanced_comment/balance_sample_comm1.csv')
training2 = pd.read_csv('/content/drive/MyDrive/reddit/balanced_comment/balance_sample_comm2.csv')
balanced_sample = pd.concat([training1,training2], ignore_index = True)
balanced_sample = balanced_sample[balanced_sample['total_no_drug'].notnull()]

del training1
del training2



<ipython-input-3-ac8f7fd4c4c2>:3: DtypeWarning: Columns (5,7,9,11,13,15,17,19) have mixed types. Specify dtype option on import or set low_memory=False.
  training1 = pd.read_csv('/content/drive/MyDrive/reddit/balanced_comment/balance_sample_comm1.csv')
<ipython-input-3-ac8f7fd4c4c2>:4: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('/content/drive/MyDrive/reddit/balanced_comment/balance_sample_comm2.csv')


In [4]:

reduction_percentage = 0.60
n_noD =  round(len(balanced_sample[balanced_sample['match'] == 0]) * reduction_percentage)
n_D = round(len(balanced_sample[balanced_sample['match'] == 1]) * reduction_percentage)

balanced_sample1 = balanced_sample[balanced_sample['match'] == 1].sample(n=n_D, random_state=1)
balanced_sample2 = balanced_sample[balanced_sample['match'] == 0].sample(n=n_noD, random_state=1)

balanced_sample = pd.concat([balanced_sample1,balanced_sample2])

del balanced_sample1
del balanced_sample2
len(balanced_sample)

1058415

In [7]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D

# Check for GPU availability
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("No GPU detected. Using CPU.")

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

# Define the input layers
input_ids = Input(shape=(None,), dtype=tf.int32, name='input_ids')
attention_mask = Input(shape=(None,), dtype=tf.int32, name='attention_mask')

# Generate BERT embeddings
bert_output = bert_model(input_ids, attention_mask=attention_mask)[0]

# Add additional layers for classification
x = GlobalAveragePooling1D()(bert_output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.1)(x)
output = Dense(1, activation='sigmoid')(x)

# Build the model
model = Model(inputs=[input_ids, attention_mask], outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()
def preprocess_data(texts, labels, max_length=128):
    # Tokenize input texts
    input_ids = []
    attention_masks = []

    for text in texts:
        encoding = tokenizer(text, truncation=True, padding='max_length', max_length=max_length, return_tensors='tf')
        input_ids.append(encoding['input_ids'][0])
        attention_masks.append(encoding['attention_mask'][0])

    return {
        'input_ids': tf.convert_to_tensor(input_ids),
        'attention_mask': tf.convert_to_tensor(attention_masks),
        'labels': tf.convert_to_tensor(labels, dtype=tf.float32)
    }

def train_model(train_texts, train_labels, batch_size=32, epochs=3):
    # Preprocess training data
    train_data = preprocess_data(train_texts, train_labels)

    # Train the model
    model.fit(
        x={'input_ids': train_data['input_ids'], 'attention_mask': train_data['attention_mask']},
        y=train_data['labels'],
        batch_size=batch_size,
        epochs=epochs,
        validation_split=0.2  # You can adjust the validation split as needed
    )


# Check for GPU availability and print details
physical_devices = tf.config.list_physical_devices('GPU')
#if physical_devices:
#    tf.config.experimental.set_memory_growth(physical_devices[0], True)
#    print(f"Using GPU: {physical_devices[0]}")

# Example usage with custom batch size and epochs
train_texts = balanced_sample['total_no_drug'].astype(str).tolist()
train_labels = balanced_sample['match'].tolist()

# Preprocess and train on GPU
with tf.device('/GPU:0'):
    # Set custom batch size and epochs
    custom_batch_size = 64
    custom_epochs = 4

    train_model(train_texts, train_labels, batch_size=custom_batch_size, epochs=custom_epochs)


Default GPU Device: /device:GPU:0


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, None)]               0         []                            
                                                                                                  
 attention_mask (InputLayer  [(None, None)]               0         []                            
 )                                                                                                
                                                                                                  
 tf_bert_model_2 (TFBertMod  TFBaseModelOutputWithPooli   1094822   ['input_ids[0][0]',           
 el)                         ngAndCrossAttentions(last_   40         'attention_mask[0][0]']      
                             hidden_state=(None, None,                                      

KeyboardInterrupt: 

In [ ]:

balanced_sample_multi = balanced_sample[balanced_sample['match'] == 1]
drug_summary = balanced_sample_multi.columns.tolist()
count_columns = [col for col in drug_summary if col.startswith('count_')]

# Create the 'labels' column
balanced_sample_multi['labels'] = balanced_sample_multi.apply(
    lambda row: [col for col in count_columns if row[col] > 0],
    axis=1
)
len(balanced_sample_multi)

<ipython-input-9-3e81ea3764d8>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  balanced_sample_multi['labels'] = balanced_sample_multi.apply(


459373

In [ ]:
#binary code

import os
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, recall_score
import pandas as pd


class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.encodings = tokenizer(texts, return_tensors='pt', max_length=max_length, padding='max_length', truncation=True)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {'input_ids': self.encodings['input_ids'][idx], 'attention_mask': self.encodings['attention_mask'][idx], 'label': self.labels[idx]}

class BERTClassifier(nn.Module):
    def __init__(self, bert_model_name, num_classes):
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.dropout(pooled_output)
        logits = self.fc(x)
        return logits

def train(model, data_loader, optimizer, scheduler, device):
    model.train()
    for batch in data_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = nn.CrossEntropyLoss()(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()

def evaluate(model, data_loader, device):
    model.eval()
    predictions = []
    actual_labels = []
    with torch.inference_mode():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs, dim=1)
            predictions.extend(preds.cpu().tolist())
            actual_labels.extend(labels.cpu().tolist())
    return recall_score(actual_labels, predictions), classification_report(actual_labels, predictions)



In [ ]:
bert_model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(bert_model_name, cache_dir='/content/')


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:

# Set up parameters
#bert_model_name = 'bert-base-uncased'
num_classes = 2
max_length = 128
batch_size = 1000
num_epochs = 4
learning_rate = 2e-5

# Loading and splitting the data
texts = balanced_sample['total_no_drug'].astype(str).tolist()
labels = balanced_sample['match'].tolist()

train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Initialize tokenizer, dataset, and data loader
bert_model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(bert_model_name, cache_dir='/content/')

#train_dataloader = DataLoader(TextClassificationDataset(train_texts, train_labels, tokenizer, max_length), batch_size=batch_size, shuffle=True)
#val_dataloader = DataLoader(TextClassificationDataset(val_texts, val_labels, tokenizer, max_length), batch_size=batch_size)

train_dataset = TextClassificationDataset(train_texts, train_labels, tokenizer, max_length)
val_dataset = TextClassificationDataset(val_texts, val_labels, tokenizer, max_length)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, num_workers=2)



# Set up the device and model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BERTClassifier(bert_model_name, num_classes).to(device)

# Set up optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=learning_rate)
total_steps = len(train_dataloader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Training the model
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    train(model, train_dataloader, optimizer, scheduler, device)
    recall, report = evaluate(model, val_dataloader, device)
    print(f"Validation recall: {recall:.4f}")
    print(report)

# Saving the final model
torch.save(model.state_dict(), "/content/drive/MyDrive/reddit/balanced_comment/bert_classifier_bin.pth")



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
#binary code

import os
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, recall_score
import pandas as pd


class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.encodings = tokenizer(texts, return_tensors='pt', max_length=max_length, padding='max_length', truncation=True)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {'input_ids': self.encodings['input_ids'][idx], 'attention_mask': self.encodings['attention_mask'][idx], 'label': self.labels[idx]}

class BERTClassifier(nn.Module):
    def __init__(self, bert_model_name, num_classes):
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.dropout(pooled_output)
        logits = self.fc(x)
        return logits

def train(model, data_loader, optimizer, scheduler, device):
    model.train()
    for batch in data_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = nn.CrossEntropyLoss()(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()

def evaluate(model, data_loader, device):
    model.eval()
    predictions = []
    actual_labels = []
    with torch.inference_mode():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs, dim=1)
            predictions.extend(preds.cpu().tolist())
            actual_labels.extend(labels.cpu().tolist())
    return recall_score(actual_labels, predictions), classification_report(actual_labels, predictions)


# Set up parameters
#bert_model_name = 'bert-base-uncased'
num_classes = 2
max_length = 128
batch_size = 1000
num_epochs = 4
learning_rate = 2e-5

# Loading and splitting the data
texts = balanced_sample['total_no_drug'].astype(str).tolist()
labels = balanced_sample['match'].tolist()

train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Initialize tokenizer, dataset, and data loader
bert_model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(bert_model_name, cache_dir='/content/')

#train_dataloader = DataLoader(TextClassificationDataset(train_texts, train_labels, tokenizer, max_length), batch_size=batch_size, shuffle=True)
#val_dataloader = DataLoader(TextClassificationDataset(val_texts, val_labels, tokenizer, max_length), batch_size=batch_size)

train_dataset = TextClassificationDataset(train_texts, train_labels, tokenizer, max_length)
val_dataset = TextClassificationDataset(val_texts, val_labels, tokenizer, max_length)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, num_workers=4)



# Set up the device and model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BERTClassifier(bert_model_name, num_classes).to(device)

# Set up optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=learning_rate)
total_steps = len(train_dataloader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Training the model
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    train(model, train_dataloader, optimizer, scheduler, device)
    recall, report = evaluate(model, val_dataloader, device)
    print(f"Validation recall: {recall:.4f}")
    print(report)

# Saving the final model
torch.save(model.state_dict(), "/content/drive/MyDrive/reddit/balanced_comment/bert_classifier_bin.pth")



In [ ]:
#multi


import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.preprocessing import MultiLabelBinarizer
import pandas as pd
import numpy as np

class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.encodings = tokenizer(texts, return_tensors='pt', max_length=max_length, padding='max_length', truncation=True)
        self.labels = torch.tensor(labels, dtype=torch.float32)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {'input_ids': self.encodings['input_ids'][idx], 'attention_mask': self.encodings['attention_mask'][idx], 'label': self.labels[idx]}

class BERTClassifier(nn.Module):
    def __init__(self, bert_model_name, num_classes):
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.dropout(pooled_output)
        logits = self.fc(x)
        return logits

def train(model, data_loader, optimizer, scheduler, device):
    model.train()
    for batch in data_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = nn.BCEWithLogitsLoss()(outputs, labels)  # Binary Cross-Entropy Loss for multilabel classification

        loss.backward()
        optimizer.step()
        scheduler.step()

def evaluate(model, data_loader, device):
    model.eval()
    predictions = []
    actual_labels = []
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            predictions.extend(torch.sigmoid(outputs).cpu().numpy())
            actual_labels.extend(labels.cpu().numpy())

    # Convert predictions and actual_labels to multilabel format
    predictions = (np.array(predictions) > 0.5).astype(int)
    actual_labels = np.array(actual_labels)

    # Calculate multilabel metrics
    recall = recall_score(actual_labels, predictions, average='weighted')
    precision = precision_score(actual_labels, predictions, average='weighted')
    f1 = f1_score(actual_labels, predictions, average='weighted')

    return recall, precision, f1

# Load your data (assuming df2 is your DataFrame)
texts = balanced_sample_multi['total_no_drug'].astype(str).tolist()

labels = balanced_sample_multi['labels'].tolist()
mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(labels)

# Set up parameters
bert_model_name = 'bert-base-uncased'
num_classes = 8
max_length = 150
batch_size = 100
num_epochs = 4
learning_rate = 2e-5

# Split the data
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Initialize tokenizer, dataset, and data loader
tokenizer = BertTokenizer.from_pretrained(bert_model_name)
train_dataset = TextClassificationDataset(train_texts, train_labels, tokenizer, max_length)
val_dataset = TextClassificationDataset(val_texts, val_labels, tokenizer, max_length)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

# Set up the device and model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BERTClassifier(bert_model_name, num_classes).to(device)

# Set up optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=learning_rate)
total_steps = len(train_dataloader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Training the model
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    train(model, train_dataloader, optimizer, scheduler, device)
    recall, precision, f1 = evaluate(model, val_dataloader, device)
    print(f"Validation recall: {recall:.4f}, precision: {precision:.4f}, f1: {f1:.4f}")

# Save the final model
torch.save(model.state_dict(), "/content/drive/MyDrive/reddit/balanced_comment/bert_classifier_multi.pth")



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/4


In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, recall_score
from torch.cuda.amp import autocast, GradScaler
import pandas as pd

# Define the dataset class
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.encodings = tokenizer(texts, return_tensors='pt', max_length=max_length, padding='max_length', truncation=True)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {'input_ids': self.encodings['input_ids'][idx], 'attention_mask': self.encodings['attention_mask'][idx], 'label': self.labels[idx]}

# Define the BERT classifier model
class BERTClassifier(nn.Module):
    def __init__(self, bert_model_name, num_classes):
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.dropout(pooled_output)
        logits = self.fc(x)
        return logits

# Define the training function
def train(model, data_loader, optimizer, scheduler, device, scaler, accumulation_steps):
    model.train()
    total_loss = 0.0
    for i, batch in enumerate(data_loader):
        optimizer.zero_grad()
        with autocast():
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            loss = nn.CrossEntropyLoss()(outputs, labels)
            total_loss += loss.item()

        scaler.scale(loss).backward()

        if (i + 1) % accumulation_steps == 0 or i == len(data_loader) - 1:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            scheduler.step()

    return total_loss / len(data_loader)


from sklearn.metrics import accuracy_score, classification_report, recall_score, precision_score, f1_score

def evaluate(model, data_loader, device):
    model.eval()
    predictions = []
    actual_labels = []
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            predictions.extend(torch.argmax(outputs, dim=1).cpu().numpy())
            actual_labels.extend(labels.cpu().numpy())

    # Calculate binary classification metrics
    accuracy = accuracy_score(actual_labels, predictions)
    recall = recall_score(actual_labels, predictions)
    precision = precision_score(actual_labels, predictions)
    f1 = f1_score(actual_labels, predictions)

    return accuracy, recall, precision, f1


# Set up parameters
#bert_model_name = 'bert-base-uncased'
num_classes = 2
max_length = 128
batch_size = 45
num_epochs = 4
learning_rate = 2e-5

# Loading and splitting the data
texts = balanced_sample['total_no_drug'].astype(str).tolist()
labels = balanced_sample['match'].tolist()

train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Initialize tokenizer, dataset, and data loader
bert_model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(bert_model_name, cache_dir='/content/')

#train_dataloader = DataLoader(TextClassificationDataset(train_texts, train_labels, tokenizer, max_length), batch_size=batch_size, shuffle=True)
#val_dataloader = DataLoader(TextClassificationDataset(val_texts, val_labels, tokenizer, max_length), batch_size=batch_size)

train_dataset = TextClassificationDataset(train_texts, train_labels, tokenizer, max_length)
val_dataset = TextClassificationDataset(val_texts, val_labels, tokenizer, max_length)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, num_workers=2)



# Set up the device and model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BERTClassifier(bert_model_name, num_classes).to(device)

# Set up optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=learning_rate)
total_steps = len(train_dataloader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
scaler = GradScaler()
torch.cuda.empty_cache()
# Training the model
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    avg_loss = train(model, train_dataloader, optimizer, scheduler, device, scaler, accumulation_steps=4)
    accuracy, recall, precision, f1 = evaluate(model, val_dataloader, device)
    print(f"Validation Accuracy: {accuracy:.4f}")
    print(f"Validation Recall: {recall:.4f}")
    print(f"Validation Precision: {precision:.4f}")
    print(f"Validation F1 Score: {f1:.4f}")
    print(f"Training Loss: {avg_loss:.4f}")
    print(f"Validation Recall: {recall:.4f}")

# Saving the final model
torch.save(model.state_dict(), "/content/drive/MyDrive/reddit/balanced_comment/bert_classifier_bin.pth")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/4


In [ ]:
torch.cuda.empty_cache()
torch.cuda.reset_max_memory_allocated()

/usr/local/lib/python3.10/dist-packages/torch/cuda/memory.py:329: FutureWarning: torch.cuda.reset_max_memory_allocated now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  warnings.warn(
